In [17]:
import wandb
from wandb.keras import WandbCallback
import warnings
warnings.filterwarnings('ignore')

In [1]:
import torch
print(f'PyTorch version: {torch.__version__}')
print('*'*10)
print(f'_CUDA version: ')
!nvcc --version
print('*'*10)
print(f'CUDNN version: {torch.backends.cudnn.version()}')
print(f'Available GPU devices: {torch.cuda.device_count()}')
print(f'Device Name: {torch.cuda.get_device_name()}')

PyTorch version: 2.0.0
**********
_CUDA version: 
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Wed_Feb__8_05:53:42_Coordinated_Universal_Time_2023
Cuda compilation tools, release 12.1, V12.1.66
Build cuda_12.1.r12.1/compiler.32415258_0
**********
CUDNN version: 8700
Available GPU devices: 1
Device Name: NVIDIA GeForce RTX 3070 Laptop GPU


# APC Dataset List

In [2]:
from utils import Notification
from pyabsa import AspectPolarityClassification as APC
from pyabsa import DatasetItem
# dataset = APC.APCDatasetList.TripAdvisorAttractionEn
# dataset
# Alternative Calling Dataset
dataset = DatasetItem("AttractionReviewEn", "302.TourismAceh8020")
dataset

C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(
C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:29: DeprecationWarning: Call to deprecated create function Descriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query th

[2023-05-19 13:28:21] (2.3.1) PyABSA(2.3.1): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.16.27

[New Feature] Aspect Sentiment Triplet Extraction since v2.1.0 (https://github.com/yangheng95/PyABSA/tree/v2/examples-v2/aspect_sentiment_triplet_extration)
[New Feature] Aspect CategoryOpinion Sentiment Quadruple Extraction since v2.2.0 (https://github.com/yangheng95/PyABSA/tree/v2/examples-v2/aspect_opinion_sentiment_category_extraction)



['302.TourismAceh8020']

C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\multiprocessing\pool.py:265: ResourceWarning: unclosed running multiprocessing pool <multiprocessing.pool.Pool state=RUN pool_size=1>
  _warn(f"unclosed running multiprocessing pool {self!r}",


# Training

In [7]:
# clear cache
import torch
!set CUDA_LAUNCH_BLOCKING=1
!set TORCH_USE_CUDA_DSA = 1
torch.cuda.empty_cache()
torch.backends.cuda.max_split_size_mb = 5500

C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=4>
  return process_handler(cmd, _system_body)
C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)
C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=6>
  return process_handler(cmd, _system_body)


In [8]:
# from metric_visualizer import MetricVisualizer
# MV = MetricVisualizer('fast_lsa_t_v2_AttractionReviewEn')

# config = APC.APCConfigManager.get_apc_config_english()
# config.model = APC.APCModelList.FAST_LSA_T_V2
# config.pretrained_bert = 'indobenchmark/indobert-base-p2'
# config.pretrained_bert = 'indobenchmark/indobert-base-p2'
# config.num_epoch = 30
# config.batch_size = 16
# config.max_seq_len = 90
# config.optimizer = 'adamw'
# config.dropout = 0.3
# config.lsa = True
# config.window = 'lr'
# config.learning_rate = 0.00002
# config.l2reg = 0.00001
# config.eta = 1  # control the k values
# config.eta_lr = 0.01
# config.MV = MV

# config = APC.APCConfigManager.get_apc_config_english()
# config.model = APC.APCModelList.FAST_LSA_T_V2
# # config.pretrained_bert = 'microsoft/deberta-v3-large'
# # config.pretrained_bert = 'yangheng/deberta-v3-base-absa-v1.1'

from pyabsa import ModelSaveOption, DeviceTypeOption
config = APC.APCConfigManager.get_apc_config_english()
config.num_epoch = 10
config.model = APC.APCModelList.FAST_LSA_T_V2
config.max_seq_len = 120
config.optimizer = 'adamw'
config.lcf = 'cdw'
config.cache_dataset = False
config.similarity_threshold = 1
config.dropout = 0.5
config.patience = 20
config.log_step = 5
config.SRD = 3
config.eta = 1
config.eta_lr = 0.001
config.lsa = True
config.learning_rate = 2e-5
config.batch_size = 16
config.evaluate_begin = 0
config.l2reg = 1e-5
config.cross_validate_fold = -1

In [11]:
config.keys()

dict_keys(['model', 'optimizer', 'learning_rate', 'cache_dataset', 'warmup_step', 'deep_ensemble', 'use_bert_spc', 'max_seq_len', 'patience', 'SRD', 'dlcf_a', 'dca_p', 'dca_layer', 'use_syntax_based_SRD', 'sigma', 'lcf', 'lsa', 'window', 'eta', 'eta_lr', 'dropout', 'l2reg', 'num_epoch', 'batch_size', 'initializer', 'seed', 'output_dim', 'log_step', 'dynamic_truncate', 'srd_alignment', 'evaluate_begin', 'similarity_threshold', 'cross_validate_fold', 'use_amp', 'overwrite_cache', 'pretrained_bert', 'dataset', 'from_checkpoint', 'checkpoint_save_mode', 'auto_device', 'path_to_save', 'load_aug', 'inference_model', 'device', 'device_name', 'model_name', 'hidden_dim', 'embed_dim', 'ABSADatasetsVersion', 'PyABSAVersion', 'TransformersVersion', 'TorchVersion', 'dataset_name', 'MV', 'save_mode', 'logger', 'task_code', 'task_name', 'dataset_file', 'model_path_to_save', 'inputs_cols', 'spacy_model', 'index_to_label', 'label_to_index', 'tokenizer', 'embedding_matrix', 'metrics_of_this_checkpoint',

In [19]:
wandb.init(project="pyabsa", entity="farrel-arrizal", reinit=True)
config = wandb.config

wandb: Currently logged in as: farrel-arrizal. Use `wandb login --relogin` to force relogin


In [9]:
Notification.send('Training started!')
try:
    trainer = APC.APCTrainer(
    config=config,
    dataset=dataset,
    # from_checkpoint="fast_lsa_t_v2_AttractionReviewEn_acc_91.26_f1_85.16",
    # if you want to resume training from our pretrained checkpoints, you can pass the checkpoint name here
    auto_device=DeviceTypeOption.AUTO,
    path_to_save=None,  # set a path to save checkpoints, if it is None, save checkpoints at 'checkpoints' folder
    checkpoint_save_mode=ModelSaveOption.SAVE_MODEL_STATE_DICT,
    load_aug=False,
    # there are some augmentation dataset for integrated datasets, you use them by setting load_aug=True to improve performance
    )
    # config.MV.next_trial()
    Notification.send('Training completed!')
    # wandb.finish()
except Exception as e:
    Notification.send(f'Training failed! {e}')
    # wandb.finish()
    raise e

[2023-05-19 08:54:23] (2.3.1) Set Model Device: cuda:0
[2023-05-19 08:54:23] (2.3.1) Device Name: NVIDIA GeForce RTX 3070 Laptop GPU
2023-05-19 08:54:24,294 INFO: PyABSA version: 2.3.1
2023-05-19 08:54:24,297 INFO: Transformers version: 4.28.1
2023-05-19 08:54:24,298 INFO: Torch version: 2.0.0+cuda11.8
2023-05-19 08:54:24,298 INFO: Device: NVIDIA GeForce RTX 3070 Laptop GPU
2023-05-19 08:54:24,298 INFO: AttractionReviewEn in the trainer is not a exact path, will search dataset in current working directory


C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\multiprocessing\pool.py:265: ResourceWarning: unclosed running multiprocessing pool <multiprocessing.pool.Pool state=RUN pool_size=1>
  _warn(f"unclosed running multiprocessing pool {self!r}",


2023-05-19 08:54:25,485 INFO: You can set load_aug=True in a trainer to augment your dataset (English only yet) and improve performance.


C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\site-packages\transformers\convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of the model checkpoint at yangheng/deberta-v3-base-absa-v1.1 were not used when initializing DebertaV2Model: ['pooler.dense.bias', 'classifier.bias', 'classifier.weight', 'pooler.dense.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializin

2023-05-19 08:54:32,692 INFO: Load dataset from integrated_datasets\apc_datasets\500.Tripadvisor\511.attraction_en\train.tourism_review_en.txt.apc


...

preparing dataloader: 100%|██████████| 1580/1580 [00:01<00:00, 1058.77it/s]

2023-05-19 08:54:44,795 INFO: Dataset Label Details: {'Neutral': 247, 'Positive': 1106, 'Negative': 227, 'Sum': 1580}


2023-05-19 08:54:45,464 INFO: test data examples:
 [{'ex_id': tensor(0), 'text_raw': 'it was simply great fun .', 'text_spc': '[CLS] it was simply great fun . [SEP] fun [SEP]', 'aspect': 'fun', 'aspect_position': tensor(0, dtype=torch.int32), 'lca_ids': tensor([0.7500, 0.8750, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.

Epoch:  0 | Smooth Loss: 0.5634:   5%|▍         | 19/412 [01:05<22:37,  3.45s/it, Dev Acc:84.43(max:84.43) Dev F1:72.65(max:72.65)]


KeyboardInterrupt: 

In [14]:
# visualize
save_prefix = 'report/{}_{}'.format(config.model_name, config.dataset_name)

MV.summary(save_path=save_prefix, no_print=True)  # save fig_preview into .tex and .pdf format
MV.traj_plot_by_trial(save_path=save_prefix, xlabel='', xrotation=30,
                      minorticks_on=True)  # save fig_preview into .tex and .pdf format
MV.violin_plot_by_trial(save_path=save_prefix, xticks=config.max_seq_len,
                        xlabel=r'$\eta$')  # save fig_preview into .tex and .pdf format
MV.box_plot_by_trial(save_path=save_prefix, xticks=config.max_seq_len,
                     xlabel=r'$\eta$')  # save fig_preview into .tex and .pdf format
MV.avg_bar_plot_by_trial(save_path=save_prefix, xticks=config.max_seq_len,
                         xlabel=r'$\eta$')  # save fig_preview into .tex and .pdf format
MV.sum_bar_plot_by_trial(save_path=save_prefix, xticks=config.max_seq_len,
                         xlabel=r'$\eta$')  # save fig_preview into .tex and .pdf format
MV.scott_knott_plot(save_path=save_prefix, minorticks_on=False, xticks=config.max_seq_len,
                    xlabel=r'$\eta$')  # save fig_preview into .tex and .pdf format

AttributeError: 'MetricVisualizer' object has no attribute 'traj_plot_by_trial'

## load trained model for inference

In [7]:
import warnings
warnings.filterwarnings('ignore')
from pyabsa import AspectPolarityClassification as APC
from pyabsa.tasks.AspectPolarityClassification import SentimentClassifier
# sentiment_classifier = trainer.load_trained_model()
# assert isinstance(sentiment_classifier, SentimentClassifier)

In [17]:
# find a suitable checkpoint and use the name:
sentiment_classifier = APC.SentimentClassifier(
    checkpoint='checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_90.51_f1_85.52',  # you can also use the checkpoint path here
)

[2023-05-18 22:23:32] (2.3.1) Load sentiment classifier from checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_90.51_f1_85.52
[2023-05-18 22:23:32] (2.3.1) config: checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_90.51_f1_85.52\fast_lsa_t_v2.config
[2023-05-18 22:23:32] (2.3.1) state_dict: checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_90.51_f1_85.52\fast_lsa_t_v2.state_dict
[2023-05-18 22:23:32] (2.3.1) model: None
[2023-05-18 22:23:32] (2.3.1) tokenizer: checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_90.51_f1_85.52\fast_lsa_t_v2.tokenizer
[2023-05-18 22:23:33] (2.3.1) Set Model Device: cuda:0
[2023-05-18 22:23:33] (2.3.1) Device Name: NVIDIA GeForce RTX 3070 Laptop GPU


Some weights of the model checkpoint at yangheng/deberta-v3-base-absa-v1.1 were not used when initializing DebertaV2Model: ['pooler.dense.bias', 'classifier.bias', 'pooler.dense.weight', 'classifier.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
valid_path = 'D:\\project\\PyABSA\integrated_datasets\\valid_datasets\\512.attraction_en'

In [19]:
sentiment_classifier.batch_predict(
    target_file=valid_path,  # the batch_predict() is only available for a file only, please put the examples in a file
    print_result=True,
    save_result=True,
    ignore_error=True,
    eval_batch_size=32,
)

run inference: 100%|██████████| 51/51 [00:09<00:00,  5.35it/s]


[2023-05-18 22:23:58] (2.3.1) Example 0: this <place:Positive(confidence:0.998, ref:Positive)> is certainly very popular , and it was full of locals , tourists and people pestering you to buy things . there ' s a long trail of shops leading down to the temple , which is testament to how popular it is as a tourist destination .
[2023-05-18 22:23:58] (2.3.1) Example 1: lovely , easy <walk:Positive(confidence:0.998, ref:Positive)> with great <views:Positive(confidence:0.998, ref:Positive)> . good way to spend an hour or two away from bustle of ubud . kara cafe might offer the best <views:Positive(confidence:0.998, ref:Positive)> from its first floor <terrace:Positive(confidence:0.664, ref:Neutral)> .
[2023-05-18 22:23:58] (2.3.1) Example 2: great to spend a day with <monkeys:Positive(confidence:0.997, ref:Positive)> , tropical and cool with big <trees:Positive(confidence:0.991, ref:Positive)> over hanging . plenty of setting to stop and rest .
[2023-05-18 22:23:58] (2.3.1) Example 3: nice

[{'text': "this place is certainly very popular , and it was full of locals , tourists and people pestering you to buy things . there ' s a long trail of shops leading down to the temple , which is testament to how popular it is as a tourist destination .",
  'aspect': ['place'],
  'sentiment': ['Positive'],
  'confidence': [0.9983324408531189],
  'probs': [array([2.8822152e-04, 1.3793833e-03, 9.9833244e-01], dtype=float32)],
  'ref_sentiment': ['Positive'],
  'ref_check': ['Correct'],
  'perplexity': 'N.A.'},
 {'text': 'lovely , easy walk with great views . good way to spend an hour or two away from bustle of ubud . kara cafe might offer the best views from its first floor terrace .',
  'aspect': ['walk', 'views', 'terrace'],
  'sentiment': ['Positive', 'Positive', 'Positive'],
  'confidence': [0.9978744983673096, 0.9978984594345093, 0.6640595197677612],
  'probs': [array([3.7436478e-04, 1.7512055e-03, 9.9787450e-01], dtype=float32),
   array([3.9805466e-04, 1.7035062e-03, 9.9789846e-

In [1]:
from pyabsa import ABSADatasetList

In [24]:
from pyabsa import AspectTermExtraction as ATEPC
from pyabsa import DatasetItem
from pyabsa import ModelSaveOption, DeviceTypeOption

In [26]:
config = ATEPC.ATEPCConfigManager.get_atepc_config_english()
config.model = ATEPC.ATEPCModelList.FAST_LCF_ATEPC  # improved version of LCF-ATEPC
config.batch_size = 16
config.verbose = True
dataset = DatasetItem("AttractionReviewEn", "512.attraction_en")
trainer = ATEPC.ATEPCTrainer(
    config=config,
    dataset=dataset,
    from_checkpoint="english",
    # if you want to resume training from our pretrained checkpoints, you can pass the checkpoint name here
    auto_device=DeviceTypeOption.AUTO,  # use cuda if available
    checkpoint_save_mode=ModelSaveOption.SAVE_MODEL_STATE_DICT,  # save state dict only instead of the whole model
    load_aug=False,
    # there are some augmentation dataset for integrated datasets, you use them by setting load_aug=True to improve performance
)

[2023-05-18 23:24:04] (2.3.1) Set Model Device: cuda:0
[2023-05-18 23:24:04] (2.3.1) Device Name: NVIDIA GeForce RTX 3070 Laptop GPU
2023-05-18 23:24:04,455 INFO: PyABSA version: 2.3.1
2023-05-18 23:24:04,456 INFO: Transformers version: 4.28.1
2023-05-18 23:24:04,456 INFO: Torch version: 2.0.0+cuda11.8
2023-05-18 23:24:04,457 INFO: Device: NVIDIA GeForce RTX 3070 Laptop GPU
2023-05-18 23:24:04,457 INFO: AttractionReviewEn in the trainer is not a exact path, will search dataset in current working directory
2023-05-18 23:24:05,622 INFO: You can set load_aug=True in a trainer to augment your dataset (English only yet) and improve performance.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
convert examples to features: 0it [00:00, ?it/s]

2023-05-18 23:24:09,547 INFO: Dataset Label Details: {'Sum': 0}



convert examples to features: 0it [00:00, ?it/s]

2023-05-18 23:24:10,093 INFO: Dataset Label Details: {'Sum': 0}



convert examples to features: 0it [00:00, ?it/s]

2023-05-18 23:24:10,667 INFO: Dataset Label Details: {'Sum': 0}


ValueError: num_samples should be a positive integer value, but got num_samples=0

## load trained model for inference

In [2]:
import warnings
warnings.filterwarnings('ignore')
from pyabsa import AspectPolarityClassification as APC
sentiment_classifier = APC.SentimentClassifier(
    checkpoint="checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_90.51_f1_85.52"
)

In [16]:
# predict
text = "everything is always cooked to perfection, the [B-ASP]service[E-ASP] is excellent, the [B-ASP]decor[E-ASP] cool and understated."
result = sentiment_classifier.predict(text=text, print_result=True)
result

[2023-05-22 12:58:34] (2.3.1) Warning: reference sentiment does not exist or its number 0 is not equal to aspect number 2, text: everything is always cooked to perfection, the [B-ASP]service[E-ASP] is excellent, the [B-ASP]decor[E-ASP] cool and understated.
[2023-05-22 12:58:34] (2.3.1) Warning: reference sentiment does not exist or its number 0 is not equal to aspect number 2, text: everything is always cooked to perfection, the [B-ASP]service[E-ASP] is excellent, the [B-ASP]decor[E-ASP] cool and understated.
[2023-05-22 12:58:34] (2.3.1) Example 0: everything is always cooked to perfection, the <service:Positive(confidence:0.998, ref:-100)> is excellent, the <decor:Positive(confidence:0.998, ref:-100)> cool and understated.


{'text': 'everything is always cooked to perfection, the service is excellent, the decor cool and understated.',
 'aspect': ['service', 'decor'],
 'sentiment': ['Positive', 'Positive'],
 'confidence': [0.9976184964179993, 0.9979497790336609],
 'probs': [array([3.7827934e-04, 2.0031740e-03, 9.9761850e-01], dtype=float32),
  array([3.8940442e-04, 1.6608820e-03, 9.9794978e-01], dtype=float32)],
 'ref_sentiment': ['-100', '-100'],
 'ref_check': ['', ''],
 'perplexity': 'N.A.'}

In [17]:
dict = {
    'text': result['text'],
    'aspects': result['aspect'],
    'sentiment': result['sentiment'], 'confidence': result['confidence']}
dict

{'text': 'everything is always cooked to perfection, the service is excellent, the decor cool and understated.',
 'aspects': ['service', 'decor'],
 'sentiment': ['Positive', 'Positive'],
 'confidence': [0.9976184964179993, 0.9979497790336609]}

In [1]:
import warnings
warnings.filterwarnings('ignore')
from pyabsa import AspectPolarityClassification as APC
sentiment_classifier = APC.SentimentClassifier(
    checkpoint="checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_90.51_f1_85.52"
)

C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(
C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:29: DeprecationWarning: Call to deprecated create function Descriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query th

[2023-05-26 04:19:28] (2.3.1) PyABSA(2.3.1): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.16.27

[New Feature] Aspect Sentiment Triplet Extraction since v2.1.0 (https://github.com/yangheng95/PyABSA/tree/v2/examples-v2/aspect_sentiment_triplet_extration)
[New Feature] Aspect CategoryOpinion Sentiment Quadruple Extraction since v2.2.0 (https://github.com/yangheng95/PyABSA/tree/v2/examples-v2/aspect_opinion_sentiment_category_extraction)

[2023-05-26 04:19:39] (2.3.1) Load sentiment classifier from checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_90.51_f1_85.52
[2023-05-26 04:19:39] (2.3.1) config: checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_90.51_f1_85.52\fast_lsa_t_v2.config
[2023-05-26 04:19:39] (2.3.1) state_dict: checkpoints/fast_lsa_t_v2_AttractionReviewEn_acc_90.51_f1_85.52\fast_lsa_t_v2.state_dict
[2023-05-26 04:19:39] (2.3.1) model: None
[2023-05-26 04:19:39] (2

C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\multiprocessing\pool.py:265: ResourceWarning: unclosed running multiprocessing pool <multiprocessing.pool.Pool state=RUN pool_size=1>
  _warn(f"unclosed running multiprocessing pool {self!r}",


[2023-05-26 04:19:39] (2.3.1) Set Model Device: cuda:0
[2023-05-26 04:19:39] (2.3.1) Device Name: NVIDIA GeForce RTX 3070 Laptop GPU


C:\Users\LENOVO\anaconda3\envs\PyABSA\lib\site-packages\transformers\convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of the model checkpoint at yangheng/deberta-v3-base-absa-v1.1 were not used when initializing DebertaV2Model: ['classifier.weight', 'classifier.bias', 'pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializin

In [3]:
# batch predict on training to check acc training
sentiment_classifier.batch_predict(
    target_file='D:\\project\\PyABSA\\integrated_datasets\\apc_datasets\\500.Tripadvisor\\511.attraction_en\\train.tourism_review_en.txt.apc.inference',  # the batch_predict() is only available for a file only, please put the examples in a file
    print_result=True,
    save_result=True,
    ignore_error=True,
)

run inference: 100%|██████████| 206/206 [00:43<00:00,  4.71it/s]


[2023-05-26 04:22:27] (2.3.1) Example 0: honestly this <place:Positive(confidence:0.997, ref:Positive)> is magical ! i mean the river , the huge trees , the crazy monkeys asking for food . it is just too much to take in . i absolutely loved it .
[2023-05-26 04:22:27] (2.3.1) Example 1: the <terrace:Positive(confidence:0.997, ref:Positive)> itself is beautiful but it has surrounded by hundreds and hundreds of shops and the street is very narrow and congested due to traffic , all this kills the beauty and charm of tegalalang rice <terrace:Positive(confidence:0.997, ref:Positive)> .
[2023-05-26 04:22:27] (2.3.1) Example 2: it is really cool , lovely <place:Positive(confidence:0.998, ref:Positive)> to visit , lucky the weather is not too hot , lot of monkeys running around , very friendly , i am so lucky meet the king of the monkeys also the grand mother too . , very safe and clean <place:Positive(confidence:0.998, ref:Positive)> to visit , i have one baby set on my head over 5 mins ,
[202

[{'text': 'honestly this place is magical ! i mean the river , the huge trees , the crazy monkeys asking for food . it is just too much to take in . i absolutely loved it .',
  'aspect': ['place'],
  'sentiment': ['Positive'],
  'confidence': [0.9974724650382996],
  'probs': [array([8.5504749e-04, 1.6723830e-03, 9.9747247e-01], dtype=float32)],
  'ref_sentiment': ['Positive'],
  'ref_check': ['Correct'],
  'perplexity': 'N.A.'},
 {'text': 'the terrace itself is beautiful but it has surrounded by hundreds and hundreds of shops and the street is very narrow and congested due to traffic , all this kills the beauty and charm of tegalalang rice terrace .',
  'aspect': ['terrace', 'tegalalang rice terrace'],
  'sentiment': ['Positive', 'Negative'],
  'confidence': [0.9974125027656555, 0.9987024068832397],
  'probs': [array([9.6655556e-04, 1.6209886e-03, 9.9741250e-01], dtype=float32),
   array([9.9870241e-01, 9.0366759e-04, 3.9392605e-04], dtype=float32)],
  'ref_sentiment': ['Positive', 'Ne